# 신용정보 EDA

In [ ]:
import sys
sys.path.append('../utils')

In [2]:
from sklearn.pipeline import Pipeline
from sklearn import set_config
from preprocessing import (
    DateElapsedTransformer, NumericFeaturePreprocessor, ObjectFeaturePreprocessor, NumericTypeOptimizer,
)

pre_optimizer = NumericTypeOptimizer(mode='pre')
date_transformer = DateElapsedTransformer()
numeric_transformer = NumericFeaturePreprocessor()
object_transformer = ObjectFeaturePreprocessor()
post_optimizer = NumericTypeOptimizer(mode='post')

preprocessing_pipeline = Pipeline([
    ('pre_optimizer', pre_optimizer),
    ('date', date_transformer),
    ('object', object_transformer),
    ('numeric', numeric_transformer),
    ('post_optimizer', post_optimizer),
])

# Set display configuration to visualize the pipeline
set_config(display='diagram')

# Display the pipeline
preprocessing_pipeline

Pipeline(steps=[('pre_optimizer', NumericTypeOptimizer(mode='pre')),
                ('date',
                 DateElapsedTransformer(base_date=Timestamp('2019-01-01 00:00:00'),
                                        exclude_columns=['ID', 'Segment',
                                                         '기준년월'])),
                ('object',
                 ObjectFeaturePreprocessor(exclude_columns=['ID', 'Segment'])),
                ('numeric',
                 NumericFeaturePreprocessor(exclude_columns=['기준년월'])),
                ('post_optimizer', NumericTypeOptimizer())])

In [3]:
from sklearn.preprocessing import LabelEncoder
from data_loader import (
    load_data,
    load_segment,
)

credit_df, credit_test_df = load_data("2.신용정보")

X = credit_df.drop(columns=['ID', '기준년월'])
X = preprocessing_pipeline.fit_transform(X)
X_test = preprocessing_pipeline.transform(credit_test_df.drop(columns=['ID', '기준년월']))

y = load_segment()
le = LabelEncoder()
y_encoded = le.fit_transform(y)

✅ File: ../../dataset/train/2.신용정보\201807_train_신용정보.parquet Completed!
✅ File: ../../dataset/train/2.신용정보\201808_train_신용정보.parquet Completed!
✅ File: ../../dataset/train/2.신용정보\201809_train_신용정보.parquet Completed!
✅ File: ../../dataset/train/2.신용정보\201810_train_신용정보.parquet Completed!
✅ File: ../../dataset/train/2.신용정보\201811_train_신용정보.parquet Completed!
✅ File: ../../dataset/train/2.신용정보\201812_train_신용정보.parquet Completed!
🔹 Shape : (2400000, 42)

✅ File: ../../dataset/test/2.신용정보\201807_test_신용정보.parquet Completed!
✅ File: ../../dataset/test/2.신용정보\201808_test_신용정보.parquet Completed!
✅ File: ../../dataset/test/2.신용정보\201809_test_신용정보.parquet Completed!
✅ File: ../../dataset/test/2.신용정보\201810_test_신용정보.parquet Completed!
✅ File: ../../dataset/test/2.신용정보\201811_test_신용정보.parquet Completed!
✅ File: ../../dataset/test/2.신용정보\201812_test_신용정보.parquet Completed!
🔹 Shape : (600000, 42)
Numeric Type Optimizer Transforming...
🧠 [mode=pre] 메모리 최적화: 1497.01 MB → 1098.76 MB (26.6% 감소)
Date

## Feature Selection

In [4]:
from feature import FeatureSelector

feature_engineering_pipeline = Pipeline([
    ('feature_selector', FeatureSelector()),
    ('optimize', post_optimizer),
])

X = feature_engineering_pipeline.fit_transform(X, y_encoded)
X_test = feature_engineering_pipeline.transform(X_test)

🎯 Fitting RandomForest for Feature Selection...
✅ Selected 12 features out of 42
📊 Top Selected Features by Importance:
1) 	카드이용한도금액 (0.1242)
2) 	CA한도금액 (0.0538)
3) 	일시상환론한도금액 (0.0298)
4) 	월상환론한도금액 (0.1510)
5) 	CA이자율_할인전 (0.0829)
6) 	CL이자율_할인전 (0.0345)
7) 	RV일시불이자율_할인전 (0.0433)
8) 	RV현금서비스이자율_할인전 (0.0506)
9) 	상향가능한도금액 (0.0382)
10) 	상향가능CA한도금액 (0.0437)
11) 	카드이용한도금액_B1M (0.1107)
12) 	카드이용한도금액_B2M (0.0921)


c:\Users\Lee JinSeoung\OneDrive\Desktop\4학년 1학기\고급프로그래밍\과제\Credit-Card-Segment-Classfication\.venv\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Numeric Type Optimizer Transforming...


c:\Users\Lee JinSeoung\OneDrive\Desktop\4학년 1학기\고급프로그래밍\과제\Credit-Card-Segment-Classfication\.venv\lib\site-packages\pandas\core\dtypes\astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
c:\Users\Lee JinSeoung\OneDrive\Desktop\4학년 1학기\고급프로그래밍\과제\Credit-Card-Segment-Classfication\.venv\lib\site-packages\pandas\core\dtypes\astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
c:\Users\Lee JinSeoung\OneDrive\Desktop\4학년 1학기\고급프로그래밍\과제\Credit-Card-Segment-Classfication\.venv\lib\site-packages\pandas\core\dtypes\astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
c:\Users\Lee JinSeoung\OneDrive\Desktop\4학년 1학기\고급프로그래밍\과제\Credit-Card-Segment-Classfication\.venv\lib\site-packages\pandas\core\dtypes\astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
c:\Users\Lee JinSeoung\OneDrive\Desktop\4학년 1학기\고급프로그래밍\과제\Credi

🧠 [mode=post] 메모리 최적화: 219.73 MB → 54.93 MB (75.0% 감소)
Numeric Type Optimizer Transforming...
🧠 [mode=post] 메모리 최적화: 54.93 MB → 13.73 MB (75.0% 감소)


c:\Users\Lee JinSeoung\OneDrive\Desktop\4학년 1학기\고급프로그래밍\과제\Credit-Card-Segment-Classfication\.venv\lib\site-packages\pandas\core\dtypes\astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
c:\Users\Lee JinSeoung\OneDrive\Desktop\4학년 1학기\고급프로그래밍\과제\Credit-Card-Segment-Classfication\.venv\lib\site-packages\pandas\core\dtypes\astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
c:\Users\Lee JinSeoung\OneDrive\Desktop\4학년 1학기\고급프로그래밍\과제\Credit-Card-Segment-Classfication\.venv\lib\site-packages\pandas\core\dtypes\astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
c:\Users\Lee JinSeoung\OneDrive\Desktop\4학년 1학기\고급프로그래밍\과제\Credit-Card-Segment-Classfication\.venv\lib\site-packages\pandas\core\dtypes\astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
c:\Users\Lee JinSeoung\OneDrive\Desktop\4학년 1학기\고급프로그래밍\과제\Credi

In [5]:
import pandas as pd

train_df = pd.concat([credit_df[['ID', '기준년월']], X], axis=1)
test_df = pd.concat([credit_test_df[['ID', '기준년월']], X_test], axis=1)

train_df.to_parquet('../../dataset/train/credit.parquet', index=False)
test_df.to_parquet('../../dataset/test/credit.parquet', index=False)